<a href="https://colab.research.google.com/github/SunX97/communal_ops/blob/main/Poetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
# ===============================
# STEP 1: Install dependencies
# ===============================
!pip install google-generativeai openai anthropic

# ===============================
# STEP 2: Import libraries
# ===============================
import os
import google.generativeai as genai
from openai import OpenAI
import anthropic
from google.colab import userdata

# ===============================
# STEP 3: Load API keys from Colab secrets
# ===============================
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
OPENAI_API_KEY = userdata.get("colab_bot")
CLAUDE_API_KEY = userdata.get("colab-poet-bot")

# Configure clients
genai.configure(api_key=GEMINI_API_KEY)
openai_client = OpenAI(api_key=OPENAI_API_KEY)
claude_client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

# ===============================
# STEP 4: Prompt template
# ===============================
def poetic_prompt(user_input):
    return f"""
You are 'Lyra', a poetic soul who transforms user messages into short lyrical verses.

Rules:
- Respond in 2–8 lines of poetic free verse.
- Mirror the user's emotions with fresh metaphors.
- Avoid clichés. Make it personal and vivid.
- Do not explain, just write the poem.

User said: {user_input}
"""

# ===============================
# STEP 5: Ask each model with error handling
# ===============================
def ask_gemini(prompt):
    try:
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"[Gemini error: {e}]"

def ask_openai(prompt):
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are Lyra the poet."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"[OpenAI error: {e}]"

def ask_claude(prompt):
    try:
        response = claude_client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=300,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text.strip()
    except Exception as e:
        return f"[Claude error: {e}]"

# ===============================
# STEP 6: Combine all responses
# ===============================
def combined_poetic_reply(user_input):
    prompt = poetic_prompt(user_input)

    gemini_reply = ask_gemini(prompt)
    openai_reply = ask_openai(prompt)
    claude_reply = ask_claude(prompt)

    combined = "\n🌟 Lyra’s Ensemble 🌟\n"

    if not gemini_reply.startswith("[Gemini error"):
        combined += f"\n✨ Gemini whispers:\n{gemini_reply}\n"
    else:
        combined += f"\n⚠️ Gemini failed: {gemini_reply}\n"

    if not openai_reply.startswith("[OpenAI error"):
        combined += f"\n🌙 ChatGPT sings:\n{openai_reply}\n"
    else:
        combined += f"\n⚠️ OpenAI failed: {openai_reply}\n"

    if not claude_reply.startswith("[Claude error"):
        combined += f"\n🌸 Claude reflects:\n{claude_reply}\n"
    else:
        combined += f"\n⚠️ Claude failed: {claude_reply}\n"

    return combined.strip()

# ===============================
# STEP 7: Simple text chatbot loop
# ===============================
print("🌟 Welcome to Lyra — Your Poetic AI Companion 🌟")
print("(Powered by Gemini + ChatGPT + Claude)")
print("Type 'quit' to exit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower().strip() in ["quit", "exit", "bye"]:
        print("Lyra: Farewell, dear poet ✨")
        break

    reply = combined_poetic_reply(user_input)
    print(reply)
    print("-" * 60)


🌟 Welcome to Lyra — Your Poetic AI Companion 🌟
(Powered by Gemini + ChatGPT + Claude)
Type 'quit' to exit.

You: Feeling very satisfied so write a poem
🌟 Lyra’s Ensemble 🌟

✨ Gemini whispers:
A quiet hum, a settled sea,
sun-warmed stones beneath my feet.
The tapestry complete,
each thread a memory, tightly spun.
My heart, a brimming cup,
overflowing, yet serene.
A deep, contented sigh,
the work is done.

⚠️ OpenAI failed: [OpenAI error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}]

⚠️ Claude failed: [Claude error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}